# Trying out GNN using graph convolutional network architecture

In [1]:
!pip install torch
!pip install torch_geometric

In [1]:
from torch_geometric.datasets import Planetoid

In [2]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
graph = dataset[0]

In [3]:
graph

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [4]:
graph.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [5]:
graph.x.shape

torch.Size([2708, 1433])

#### To visualise the data

In [6]:
import random
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

In [7]:
def convert_to_networkx(graph, n_sample=None):

    g = to_networkx(graph, node_attrs=["x"])
    y = graph.y.numpy()

    if n_sample is not None:
        sampled_nodes = random.sample(g.nodes, n_sample)
        g = g.subgraph(sampled_nodes)
        y = y[sampled_nodes]

    return g, y

In [8]:
def plot_graph(g, y):

    plt.figure(figsize=(9, 7))
    nx.draw_spring(g, node_size=30, arrows=False, node_color=y)
    plt.show() 

In [1]:
g, y = convert_to_networkx(graph, n_sample=1000)
plot_graph(g, y)

NameError: name 'convert_to_networkx' is not defined

In [9]:
import torch_geometric.transforms as T

#### Split into train and test datasets

In [10]:
split = T.RandomNodeSplit(num_val=0.1, num_test=0.2)
graph = split(graph)

In [11]:
graph

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [12]:
graph.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [15]:
graph.x.shape

torch.Size([2708, 1433])

#### Declare the basic MLP in PyTorch to which we shall feed the graph embeddings

In [16]:
import torch
import torch.nn as nn

In [17]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
        nn.Linear(dataset.num_node_features, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, dataset.num_classes)
        )

    def forward(self, data):
        x = data.x  # only using node features (x)
        output = self.layers(x)
        return output

#### Declare the training and testing function for the base MLP

In [18]:
def eval_node_classifier(model, graph, mask):

    model.eval()
    pred = model(graph).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())

    return acc

In [19]:
def train_node_classifier(model, graph, optimizer, criterion, n_epochs=200):

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph)
        loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
        loss.backward()
        optimizer.step()

        pred = out.argmax(dim=1)
        acc = eval_node_classifier(model, graph, graph.val_mask)

        if epoch % 10 == 0:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val Acc: {acc:.3f}')

    return model

#### Lets simply use the X values of just words and run the normal feed forward MLP to see the accuracy

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mlp = MLP().to(device)
optimizer_mlp = torch.optim.Adam(mlp.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [21]:
mlp = train_node_classifier(mlp, graph, optimizer_mlp, criterion, n_epochs=150)

Epoch: 010, Train Loss: 0.934, Val Acc: 0.679
Epoch: 020, Train Loss: 0.103, Val Acc: 0.738
Epoch: 030, Train Loss: 0.018, Val Acc: 0.705
Epoch: 040, Train Loss: 0.012, Val Acc: 0.701
Epoch: 050, Train Loss: 0.013, Val Acc: 0.701
Epoch: 060, Train Loss: 0.012, Val Acc: 0.708
Epoch: 070, Train Loss: 0.010, Val Acc: 0.708
Epoch: 080, Train Loss: 0.010, Val Acc: 0.712
Epoch: 090, Train Loss: 0.009, Val Acc: 0.716
Epoch: 100, Train Loss: 0.008, Val Acc: 0.716
Epoch: 110, Train Loss: 0.008, Val Acc: 0.720
Epoch: 120, Train Loss: 0.007, Val Acc: 0.723
Epoch: 130, Train Loss: 0.007, Val Acc: 0.727
Epoch: 140, Train Loss: 0.007, Val Acc: 0.727
Epoch: 150, Train Loss: 0.007, Val Acc: 0.723


In [22]:
output_for_mlp = mlp(graph).argmax(dim = 1)

In [23]:
mask_for_mlp = graph.test_mask

correct_preds_for_mlp = (output_for_mlp[mask_for_mlp] == graph.y[mask_for_mlp]).sum()

In [24]:
correct_preds_for_mlp/output_for_mlp[mask_for_mlp].shape[0]

tensor(0.7491)

In [25]:
test_acc = eval_node_classifier(mlp, graph, graph.test_mask)
print(f'Test Acc: {test_acc:.3f}')

Test Acc: 0.749


In [26]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [27]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        output = self.conv2(x, edge_index)

        return output

In [28]:
gcn = GCN().to(device)
optimizer_gcn = torch.optim.Adam(gcn.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
gcn = train_node_classifier(gcn, graph, optimizer_gcn, criterion)

Epoch: 010, Train Loss: 0.889, Val Acc: 0.808
Epoch: 020, Train Loss: 0.366, Val Acc: 0.875
Epoch: 030, Train Loss: 0.223, Val Acc: 0.871
Epoch: 040, Train Loss: 0.171, Val Acc: 0.871
Epoch: 050, Train Loss: 0.147, Val Acc: 0.867
Epoch: 060, Train Loss: 0.136, Val Acc: 0.863
Epoch: 070, Train Loss: 0.128, Val Acc: 0.867
Epoch: 080, Train Loss: 0.120, Val Acc: 0.863
Epoch: 090, Train Loss: 0.113, Val Acc: 0.863
Epoch: 100, Train Loss: 0.106, Val Acc: 0.871
Epoch: 110, Train Loss: 0.101, Val Acc: 0.867
Epoch: 120, Train Loss: 0.096, Val Acc: 0.871
Epoch: 130, Train Loss: 0.092, Val Acc: 0.871
Epoch: 140, Train Loss: 0.089, Val Acc: 0.867
Epoch: 150, Train Loss: 0.086, Val Acc: 0.863
Epoch: 160, Train Loss: 0.083, Val Acc: 0.863
Epoch: 170, Train Loss: 0.081, Val Acc: 0.863
Epoch: 180, Train Loss: 0.079, Val Acc: 0.863
Epoch: 190, Train Loss: 0.077, Val Acc: 0.863
Epoch: 200, Train Loss: 0.075, Val Acc: 0.860


In [29]:
test_acc = eval_node_classifier(gcn, graph, graph.test_mask)
print(f'Test Acc: {test_acc:.3f}')

Test Acc: 0.893
